# Веб скрапинг топ 250 лучших фильмов по версии КиноПоиска

#### Совет: не запускайте данный файл, т.к. в нем не вставлены токены

## Установка пакетов

In [144]:
!pip install beautifulsoup4
!pip install scrapingant-client
!pip install kinopoisk-api-unofficial-client

## Подключение библиотек

In [145]:
from bs4 import BeautifulSoup
from scrapingant_client import ScrapingAntClient
from dataclasses import dataclass
import pandas as pd
import re
from kinopoisk_unofficial.kinopoisk_api_client import KinopoiskApiClient
from kinopoisk_unofficial.request.films.film_request import FilmRequest

## Константы

In [146]:
URL = "https://www.kinopoisk.ru"
SCRAPING_TOKEN = '<SCRAPING_TOKEN>' #https://app.scrapingant.com/
KINOPOISK_TOKEN = '<KINOPOISK_TOKEN>' #https://kinopoiskapiunofficial.tech/profile
KINOPOISK_CLIENT = KinopoiskApiClient(KINOPOISK_TOKEN) #https://pypi.org/project/kinopoisk-api-unofficial-client/

## Модели

In [147]:
@dataclass
class Film:
    #parser
    id: int
    number: int
    name: str
    rating: float #0-10
    year: int
    country: str
    genre: str
    director: str
    duration: int #minutes

## API

In [148]:
def get_film_data(id):
    try:
        request = FilmRequest(id)
        response = KINOPOISK_CLIENT.films.send_film_request(request)
        return response.film
    except:
        return None

## Парсинг

In [149]:
def parse_dynamic_website(url):
    # https://github.com/ScrapingAnt/scrapingant-client-python#api-token
    try:
        print(f'started parsing {url}')
        # Create a ScrapingAntClient instance
        client = ScrapingAntClient(token=SCRAPING_TOKEN)

        # Get the HTML page rendered content
        page_content = client.general_request(url).content
        print(f'finished parsing {url}')
        return page_content
    except:
        return ""

In [150]:
def get_movies(url: str) -> list:
    try:
        print('started getting movies')
        page_content = parse_dynamic_website(url)
        page = BeautifulSoup(page_content, 'html.parser')
        movies = page.find_all('div', class_='styles_root__ti07r')
        print('finished getting movies')
        return movies
    except:
        return []

In [151]:
movies = []
for page in range(5):
    movies += get_movies(URL+f'/lists/movies/top250/?page={page+1}')
len(movies)

started getting movies
started parsing https://www.kinopoisk.ru/lists/movies/top250/?page=1
finished parsing https://www.kinopoisk.ru/lists/movies/top250/?page=1
finished getting movies
started getting movies
started parsing https://www.kinopoisk.ru/lists/movies/top250/?page=2
finished parsing https://www.kinopoisk.ru/lists/movies/top250/?page=2
finished getting movies
started getting movies
started parsing https://www.kinopoisk.ru/lists/movies/top250/?page=3
finished parsing https://www.kinopoisk.ru/lists/movies/top250/?page=3
finished getting movies
started getting movies
started parsing https://www.kinopoisk.ru/lists/movies/top250/?page=4
finished getting movies
started getting movies
started parsing https://www.kinopoisk.ru/lists/movies/top250/?page=5
finished getting movies


150

In [152]:
dataset = []
dataset_api = []
nonBreakSpace = u'\xa0'
for movie in movies:
    try:
        id = movie.find('a', class_='base-movie-main-info_link__YwtP1')['href'].split('/')[-2]
        
        year_duration = re.sub("[^0-9| ]","",movie.find('span', class_='desktop-list-main-info_secondaryText__M_aus').text).split()
        country_genre_director = re.split('[•|:]', movie.find('span', class_='desktop-list-main-info_truncatedText__IMQRP').text.replace(nonBreakSpace, ':'))
        number = movie.find('span', class_='styles_position__TDe4E').text
        name = movie.find('span', class_='styles_mainTitle__IFQyZ').text
        rating = movie.find('span', class_='styles_kinopoiskValuePositive__vOb2E').text
        year = year_duration[0]
        country = country_genre_director[0][:-1]
        genre = country_genre_director[1][1:]
        director = country_genre_director[-1]
        duration = year_duration[1]
        film = Film(id=id, number=number, name=name, rating=rating, year=year, country=country, genre=genre, director=director, duration=duration)
        dataset.append(film)
        
        film_api = get_film_data(id)
        if film_api != None:
            dataset_api.append(film_api)
    except:
        pass
len(dataset), len(dataset_api)

(150, 150)

In [153]:
df = pd.DataFrame(data=dataset)
df

,id,number,name,rating,year,country,genre,director,duration
0,435,1,Зеленая миля,9.2,1999,США,драма,Фрэнк Дарабонт,189
1,329,2,Список Шиндлера,9.1,1993,США,драма,Стивен Спилберг,195
2,326,3,Побег из Шоушенка,9.0,1994,США,драма,Фрэнк Дарабонт,142
3,448,4,Форрест Гамп,9.0,1994,США,драма,Роберт Земекис,142
4,3498,5,Властелин колец: Возвращение короля,8.9,2003,Новая Зеландия,фэнтези,Питер Джексон,201
...,...,...,...,...,...,...,...,...,...
145,920265,146,Человек-паук: Через вселенные,8.3,2018,США,мультфильм,Боб Персичетти,117
146,740,147,Лило и Стич,8.3,2002,США,мультфильм,Дин ДеБлуа,85
147,46789,148,12 стульев,8.3,1971,СССР,комедия,Леонид Гайдай,153
148,26656,149,Коты-аристократы,8.3,1970,США,мультфильм,Вольфганг Райтерман,78


In [154]:
df_api = pd.DataFrame(data=dataset_api)
df_api

,kinopoisk_id,imdb_id,name_ru,name_en,name_original,poster_url,poster_url_preview,reviews_count,rating_good_review,rating_good_review_vote_count,...,slogan,description,short_description,editor_annotation,rating_mpaa,rating_age_limits,last_sync,serial,short_film,completed
0,435,tt0120689,Зеленая миля,None,The Green Mile,https://kinopoiskapiunofficial.tech/images/pos...,https://kinopoiskapiunofficial.tech/images/pos...,0,93.2,421,...,Пол Эджкомб не верил в чудеса. Пока не столкну...,Пол Эджкомб — начальник блока смертников в тюр...,В тюрьме для смертников появляется заключенный...,None,r,age16,2022-12-28T14:39:27.829197,False,False,False
1,329,tt0108052,Список Шиндлера,None,Schindler's List,https://kinopoiskapiunofficial.tech/images/pos...,https://kinopoiskapiunofficial.tech/images/pos...,0,90.4,349,...,Этот список - жизнь,Фильм рассказывает реальную историю загадочног...,"История немецкого промышленника, спасшего тыся...",None,r,age16,2022-12-27T18:19:00.962296,False,False,False
2,326,tt0111161,Побег из Шоушенка,None,The Shawshank Redemption,https://kinopoiskapiunofficial.tech/images/pos...,https://kinopoiskapiunofficial.tech/images/pos...,0,92.5,518,...,Страх - это кандалы. Надежда - это свобода,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Несправедливо осужденный банкир готовит побег ...,None,r,age16,2022-12-26T11:49:27.204846,False,False,False
3,448,tt0109830,Форрест Гамп,None,Forrest Gump,https://kinopoiskapiunofficial.tech/images/pos...,https://kinopoiskapiunofficial.tech/images/pos...,0,93.2,437,...,"Мир уже никогда не будет прежним, после того к...","Сидя на автобусной остановке, Форрест Гамп — н...",Полувековая история США глазами чудака из Алаб...,None,pg13,age16,2022-12-17T10:52:40.428719,False,False,False
4,3498,tt0167260,Властелин колец: Возвращение короля,None,The Lord of the Rings: The Return of the King,https://kinopoiskapiunofficial.tech/images/pos...,https://kinopoiskapiunofficial.tech/images/pos...,0,95.3,210,...,There can be no triumph without loss. No victo...,Повелитель сил тьмы Саурон направляет свою бес...,"Арагорн штурмует Мордор, а Фродо устал боротьс...",None,pg13,age12,2022-12-26T12:25:57.796351,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,920265,tt4633694,Человек-паук: Через вселенные,None,Spider-Man: Into the Spider-Verse,https://kinopoiskapiunofficial.tech/images/pos...,https://kinopoiskapiunofficial.tech/images/pos...,0,85.0,96,...,Enter a universe where more than one wears the...,"Мы всё знаем о Питере Паркере. Он спас город, ...",Пауки из разных измерений объединяются перед о...,None,pg,age6,2023-01-11T17:47:53.68435,False,False,False
146,740,tt0275847,Лило и Стич,None,Lilo & Stitch,https://kinopoiskapiunofficial.tech/images/pos...,https://kinopoiskapiunofficial.tech/images/pos...,0,91.7,22,...,He's coming to our galaxy,"Лило — это маленькая девочка, живущая на Гавай...","Одинокая Лило знакомится с монстром, который б...",None,pg,age0,2022-12-12T05:41:50.42341,False,False,False
147,46789,tt0065670,12 стульев,None,None,https://kinopoiskapiunofficial.tech/images/pos...,https://kinopoiskapiunofficial.tech/images/pos...,0,90.5,35,...,Amerika Be Seated!,Во время революции и последовавшего за ней кра...,Товарищ Бендер в погоне за бриллиантами мадам ...,None,pg13,age6,2022-12-09T06:02:10.482432,False,False,False
148,26656,tt0065421,Коты-аристократы,None,The AristoCats,https://kinopoiskapiunofficial.tech/images/pos...,https://kinopoiskapiunofficial.tech/images/pos...,0,97.7,21,...,A purr-fectly wonderful new cartoon feature,В аристократическом доме в Париже живет мама-к...,Обаятельный уличный кот помогает котятам из бл...,None,g,age6,2022-12-21T10:15:28.159309,False,False,False


In [155]:
df.to_excel(f'kinopoisk_movies_top250.xlsx', index=False)

In [156]:
df_api.to_excel(f'kinopoisk_movies_top250_API.xlsx', index=False)